#Osservazioni codice#


** spider trap**: amplio i metodi per eliminare le spidertrap

#Librerie#

In [ ]:
#url di prova
# ha sitemap-index ma non priority etc.
start_url_uno = "https://www.dragopublisher.com/it/"
# non ha sitemap
start_url_due = "https://www.wildraccoon.it/shop/"
# sito giocattolo ha tutto
start_url = "https://nonciclopedia.org/wiki/Wikipedia"

In [ ]:
!pip install rfc3986

In [ ]:
import pandas as pd
import re
from urllib.parse import urlparse
import urllib.robotparser #per gestire il file robot.txt
import xml.etree.ElementTree as ET  #per gestire i file xml
import requests #per http
import time #per gestire il tempo
from bs4 import BeautifulSoup #per gestire il parsing del html (si può usare anche per xml)
import rfc3986 # per la normalizzazione degli urls


#Fetcher#
gestione richieste e download pagine web

In [ ]:
class UserAgentPolicy:
    '''
     Class to identify user agents and store relevant information to manage their requests appropriately over time
    '''
  def __init__(self, name, base_url, path_disallow, crawl_delay, request_rate,last_access, header):
        self.base_url = base_url
        self.name = name
        self.path_disallow = path_disallow
        self.crawl_delay = crawl_delay
        self.request_rate = request_rate
        self.last_access = last_access
        self.header = header

In [ ]:
# Useragents
useragent_dict = {"Googlebot": UserAgentPolicy("Googlebot",None, None, None, None, 1e9,'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'),
                  "Bingbot": UserAgentPolicy("Bingbot",None, None, None, None, 1e9,'Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm)'),
                  "Slurp": UserAgentPolicy("Slurp",None, None, None, None, 1e9, 'Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)'),
                  "DuckDuckbot": UserAgentPolicy("DuckDuckbot",None, None, None, None, 1e9,'DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)' ),
                  "Yandex": UserAgentPolicy("Yandex",None, None, None, None, 1e9,'Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)' ),
                  "*": UserAgentPolicy("*",None, None, None, None, 1e9,'Mozilla/5.0 (compatible; MyBot/1.0; +http://example.com/bot)' )
                  }
default_agent = useragent_dict["Googlebot"]

# header generated by ChatGPT
useragent_dict_header = {'Googlebot':'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)',
                    'Bingbot':'Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm)',
                    'Slurp':'Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)',
                    'DuckDuckbot':'DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)',
                    'Yandex':'Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)',
                    '*': 'Mozilla/5.0 (compatible; MyBot/1.0; +http://example.com/bot)'}

In [ ]:
def check_robots(url,useragent=default_agent):

  '''
    Inputs:
        url (str): The base URL of the website to analyze.
        useragent (class): The user agent to use when checking access permissions. Default is '*'.

    Output:
        sitemap (list or None): List of sitemap URLs found in robots.txt, if any.

    Description:
        This function checks the robots.txt file of a given website to determine:
        - Whether the given user agent is allowed to fetch content.
        - Which paths are disallowed for the user agent.
        - The crawl delay and request rate for the user agent.
        - Any sitemap URLs provided in the robots.txt.

        It updates the given useragent object with these details.
    '''

  useragent.base_url = url

  # Initialize the robot parser
  rfl = urllib.robotparser.RobotFileParser()
  # Construct the full URL to the robots.txt file
  new_url = url + "/robots.txt"

  rfl.set_url(new_url)
  # Read and parse the robots.txt file
  rfl.read()
  check = rfl.can_fetch(useragent,new_url)
  if check:
    print("{} il sito puo effettuare il fetch dei dati".format(check))
  else:
    print("il sito non puo effettuare il fetch dei dati")

  # Retrieve the list of the path not to follow
  path_disallow = rfl.parse("Disallow")
  useragent.path_disallow = path_disallow

  # Retrieve the crawl delay (minimum interval between requests)
  crawl_delay = rfl.crawl_delay(useragent)
  useragent.crawl_delay = crawl_delay

  # Retrieve the request rate (tuple of requests per time interval)
  request_rate = rfl.request_rate(useragent)
  useragent.request_rate = request_rate

  # Retrieve any sitemap URLs declared in the robots.txt
  sitemap = rfl.site_maps()
  print("sitemap: {}".format(sitemap))

  return sitemap

In [ ]:
def check_time(useragent=default_agent):

 '''
    Inputs:
        useragent (UserAgentPolicy): The user agent object containing rate-limiting information.

    Output:
      None

    Description:
        Ensures that the user agent respects the specified crawl delay and request rate policies:
        - Calculates the appropriate delay based on the more restrictive value between `crawl_delay` and `request_rate`.
        - If neither is defined, defaults to a delay of 1.5 seconds.
        - Ensures the delay is respected by pausing execution as needed.
        - Updates `last_access` to the current time after waiting.
    '''


  # If both crawl_delay and request_rate are defined, use the more restrictive one (converted to nanoseconds)

  if useragent.request_rate and useragent.crawl_delay:
     # Calculate delay from request rate: total interval (in ns) divided by number of requests
     request_delay = (useragent.request_rate[1] * 1e9) / useragent.request_rate[0]
     # Convert crawl delay from seconds to nanoseconds
     crawl_delay = useragent.crawl_delay * 1e9
     # Use the larger (i.e., more restrictive) delay
     delay = max(request_delay, crawl_delay)

  elif useragent.crawl_delay:
     delay = useragent.crawl_delay * 1e9

  elif useragent.request_rate:
     delay = (useragent.request_rate[1] * 1e9) / useragent.request_rate[0]

  # If neither crawl_delay and request rate are defined, use a default delay of 1.5 seconds
  else:
    delay = 1.5 * 1e9  # 1.5 secondi in nanosecondi

  # Get the current time in nanoseconds
  now = time.monotonic_ns()
  wait = max(0, (useragent.last_access + delay) - now)

  if wait > 0:
     time.sleep(wait / 1e9)

  # Update last_access to the current time to enforce timing on next request
  useragent.last_access = time.monotonic_ns()


In [ ]:
def fetch(url,useragent=default_agent):
  '''
    Inputs:
        url (str): The target URL to fetch.
        useragent (UserAgentPolicy): The user agent object containing headers and rate-limiting rules.

    Outputs:
        str or None: The HTML content of the page if the request is successful (HTTP 200), otherwise None.

    Description:
        Sends an HTTP GET request to the specified URL, respecting the request policies defined for the given user agent.

    '''



  # Enforce crawl delay and request rate restrictions for the user agent
  check_time(useragent)

  # Build the request headers using the specified user agent (optional, useful for testing)
  headers = {
        'User-Agent': useragent.header
    }

  # Send the HTTP GET request
  try:
     response = requests.get(url, headers=headers, timeout=10)
     if response.status_code == 200:
       print("Pagina recuperata con successo:{}".format(url))
       html = response.text
       return html
     else:
      print("Errore nella ricezione del sito:{}".format(url))
      return None

  # Handle any request-related exceptions (connection errors, timeouts, etc.)
  except requests.RequestException as e:
        print("Eccezione durante la richiesta:", e)
        return None

# Parser#
analisi contenuto della pagina, estrazione testo e link e normalizzazione link trovati

In [ ]:
def normalize_url(url):
  '''
     Normalizza l'URL secondo lo standard RFC3986 e restituisce una stringa
  '''
  try:
      uri = rfc3986.uri_reference(url).normalize()
      return uri.unsplit()

  except Exception as e:
      print(f"Errore nella normalizzazione URL: {url} – {e}")
      return url

In [ ]:
def check_spider_traps(url):
  '''
    controllo l'url per vedere se è potenzialmente dannoso, o se spinge il crawlwe in un loop
  '''

  trap_pattern = re.compile(r"(calendar|sessionid|track|ref|sort|date=|page=\d{3,})", re.IGNORECASE)
  MAX_URL_LENGTH = 200
  MAX_PATH_DEPTH = 6

  link = urlparse(url)

  if len(link) > MAX_URL_LENGTH:
     return False

  if link.path.count('/') > MAX_PATH_DEPTH:
     return False

  if trap_pattern.search(url):
     return False

  return True


In [ ]:

def parse_sitemap(sitemap_list):
    '''
    Prende in input una lista di sitemap URLs e restituisce un DataFrame
    con: URL, priorità e frequenza di aggiornamento.
    '''
    namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
    all_entries = []

    if not sitemap_list:
        print("Nessuna sitemap fornita.")
        return pd.DataFrame(columns=['url', 'priority', 'update'])

    def parse_single_sitemap(sitemap_url):
        try:
            response = requests.get(sitemap_url, timeout=10)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Errore durante il recupero di {sitemap_url}: {e}")
            return []

        content_type = response.headers.get("Content-Type", "").lower()

        # Sitemap XML
        if "xml" in content_type:
            try:
                root = ET.fromstring(response.content)
            except ET.ParseError as e:
                print(f"Errore nel parsing XML: {e}")
                return []

            # Sitemap index
            if root.tag.endswith('index'):
                sitemap_urls = [loc.text for loc in root.findall('ns:sitemap/ns:loc', namespace)]
                entries = []
                for sub_url in sitemap_urls:
                    entries.extend(parse_single_sitemap(sub_url))
                return entries

            # Sitemap semplice
            elif root.tag.endswith('urlset'):
                return [{
                    'url': url.findtext('ns:loc', default='', namespaces=namespace),
                    'priority': url.findtext('ns:priority', default=None, namespaces=namespace),
                    'update': url.findtext('ns:changefreq', default=None, namespaces=namespace)
                } for url in root.findall('ns:url', namespace)]

        # Sitemap HTML
        elif "html" in content_type:
            soup = BeautifulSoup(response.content, "html.parser")
            return [{
                'url': normalize_url(a['href']),
                'priority': None,
                'update': None
            } for a in soup.find_all('a', href=True)]

        else:
            print(f"Formato non riconosciuto per {sitemap_url} ({content_type})")
            return []

    # Estrai tutte le voci da ogni sitemap
    for sitemap_url in sitemap_list:
        if sitemap_url:
            entries = parse_single_sitemap(sitemap_url)
            for entry in entries:
                url = normalize_url(entry['url']) #normalizzo l'url
                if check_spider_traps(url): # provo a limitare le spider-trap
                    entry['url'] = url

            all_entries.extend(entries)

    df = pd.DataFrame(all_entries, columns=['url', 'priority', 'update']).drop_duplicates()

    return df


In [ ]:
def parse_page_url(html,sitemaps_urls,useragent=default_agent):
  '''
    dato una pagina html di input cerco gli urls:
    - se sono in una directory privata li scarto
    - se gia presenti nella sitemap, sono già inidicizzate e non salvo i link (duplicati)
    - se non presenti salvo gli url permessi dal file robots.txt
  '''

  #parsing pagina html
  soup = BeautifulSoup(html, 'html.parser')

  urls = []
  # Elementi con attributo href
  for tag in soup.find_all(href=True):
      urls.append(tag['href'])

  # Elementi con attributo src
  for tag in soup.find_all(src=True):
      urls.append(tag['src'])

  # controlla se i nuovi url sono veramente accessibili per il web scraping (no in disallow)
  if useragent.path_disallow != None:
    for url in urls:
      for path in useragent.path_disallow:
        if path in url:
         urls.remove(url)

  # normalizzo gli urls
    urls = [normalize_url(url) for url in urls]

  # elimino i duplicati
    urls = list(set(urls))

  # provo a limitare le spider-trap
   # urls = [url for url in urls if check_spider_traps(url)]

  #controlla se ci sono duplicati nella sitemap se c'è
  if sitemaps_urls != None:
    def_urls = sitemaps_urls
    for url in urls:
      if url not in def_urls:
         def_urls.append(url)
  else:
    def_urls = urls

  return def_urls



In [ ]:
def parse_page_tags(html,tag_type, attrs={}):
  '''
    accetta in input una pagina html, il tipo di tag da selezionare ed eventuali attributi se necessari
  '''
  # scelgo il parser
  soup = BeautifulSoup(html, "html.parser")
  tags = soup.find_all(tag_type, attrs=attrs)

  return [str(tag) for tag in tags]

In [ ]:
def parse_page_tags_all(html,tags_type = ['p','article','section','li','h1','h2','h3']):
  '''
    accetta in input una pagina html, il tipo di tag da selezionare ed eventuali attributi se necessari
  '''
  # scelgo il parser
  soup = BeautifulSoup(html, "html.parser")
  tags = soup.find_all(tags_type) #dovrebbe rispettare l'ordine del DOM
  texts = [tag.get_text(separator=' ', strip=True) for tag in tags]

  return texts



In [ ]:
html = fetch(start_url_due, default_agent)
words = parse_page_tags_all(html)
print(words)
print(default_agent.last_access)

Pagina recuperata con successo:https://www.wildraccoon.it/shop/
['Home', 'Le birre', 'Shop Lattine Merchandising', 'Lattine', 'Merchandising', 'Il Metodo', 'Chi siamo', 'Bolli spina', 'Contatti', 'Home', 'Le birre', 'Shop Lattine Merchandising', 'Lattine', 'Merchandising', 'Il Metodo', 'Chi siamo', 'Bolli spina', 'Contatti', 'SHOP', 'Shop', 'Straight to harvest 22,00 € – 62,00 € Scegli Questo prodotto ha più varianti. Le opzioni possono essere scelte nella pagina del prodotto', 'Straight to harvest', 'Each others paranoia 20,00 € – 58,00 € Scegli Questo prodotto ha più varianti. Le opzioni possono essere scelte nella pagina del prodotto', 'Each others paranoia', 'Vfx 19,00 € – 54,00 € Scegli Questo prodotto ha più varianti. Le opzioni possono essere scelte nella pagina del prodotto', 'Vfx', 'Dolci on fire – Mango, banana & lassi 25,00 € – 68,00 € Scegli Questo prodotto ha più varianti. Le opzioni possono essere scelte nella pagina del prodotto', 'Dolci on fire – Mango, banana & lassi',

In [ ]:
sitemap = check_robots(start_url_due,default_agent)
print("sitemap")
if sitemap == None:
  urls = parse_page_url(html,None,default_agent)

allora = [url for url in urls if check_spider_traps(url)]
print(allora)

True il sito puo effettuare il fetch dei dati
sitemap: None
sitemap
['https://gmpg.org/xfn/11', 'https://www.wildraccoon.it/feed/', 'https://www.wildraccoon.it/comments/feed/', 'https://www.wildraccoon.it/shop/feed/', 'https://www.wildraccoon.it/wp-includes/css/dist/block-library/style.min.css?ver=6.8.1', 'https://www.wildraccoon.it/wp-content/plugins/cookie-notice/css/front.min.css?ver=2.5.6', 'https://www.wildraccoon.it/wp-content/plugins/woocommerce/assets/css/woocommerce-layout.css?ver=9.9.3', 'https://www.wildraccoon.it/wp-content/plugins/woocommerce/assets/css/woocommerce-smallscreen.css?ver=9.9.3', 'https://www.wildraccoon.it/wp-content/plugins/woocommerce/assets/css/woocommerce.css?ver=9.9.3', 'https://www.wildraccoon.it/wp-content/plugins/woocommerce/assets/css/brands.css?ver=9.9.3', 'https://www.wildraccoon.it/wp-content/themes/hello-elementor/assets/css/reset.css?ver=3.4.4', 'https://www.wildraccoon.it/wp-content/themes/hello-elementor/assets/css/theme.css?ver=3.4.4', 'https